## Geocoding addresses in python with Geopy
This notebook demonstrates a simple python geocoding and mapping workflow.

###  Install geocoding and mapping libraries that we will use.

- [geopy](https://geopy.readthedocs.io/), for consistant interface to different geocoding APIs
- [folium](https://github.com/python-visualization/folium), a python library for making interactive maps

In [1]:
# Geocoding library


In [2]:
!pip install geopy
!pip install folium

## Import the libraries

In [11]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import requests

## Create a geocoder that uses the OpenStreetMap Nominatim API

In [9]:
geolocator = Nominatim(timeout=10, user_agent = "denironyx@gmail.com")

## Test the geocoder with one address
location = geolocator.geocode('Barrows Hall, Berkeley CA')
location

Location(Social Sciences Building, Eshleman Road, Southside, Berkeley, Alameda County, California, 94720-1076, United States, (37.87010975, -122.25792906636786, 0.0))

## Upload a CSV file of addresses to geocode
Fetch the data to geocode with wget

In [6]:
url = "https://raw.githubusercontent.com/dlab-geo/geocoding/master/address_data/SFLandmarks.csv"
r = requests.get(url, allow_redirects=True)
open('SFLandmarks.csv', 'wb').write(r.content)

156